In [ ]:
# default_exp util

# Util

> Convenience functions.

In [ ]:
# export

import pathlib
import re
import sys
from typing import List, Dict, Union, Optional

import numpy as np
import IPython.display
import ruamel.yaml
import yaml
import pandas as pd
from pandas.core.accessor import _register_accessor as register_accessor

import gift_wrapper.core
import gift_wrapper.question
import gift_wrapper.image
# import py2gift.core
# import py2gift.question

## `render_latex`

Convenience function to print $\LaTeX$ text in Jupyter Notebook **code** (as opposed to *markdown*) cells. The purpose is to print $\LaTeX$ content that is only known at *run time*. Font size of every formula is increased (to *\Large*)

In [ ]:
# export

def render_latex(text: str) -> str:
    
    return IPython.display.Markdown(re.sub(r'\$([^\$]*)\$', '$' + '\\\Large ' + r'\1' + '$', text))

In [ ]:
render_latex('Bla blah...')

In [ ]:
render_latex('Variable $A$')

# Accessors

[Pandas](https://pandas.pydata.org/)'s [accessors mechanishm](https://pandas.pydata.org/docs/reference/api/pandas.api.extensions.register_dataframe_accessor.html) is leveraged here. In order to do that, first Pandas' `_register_accessor` method was *aliased* above as simply `register_accessor`. Then a new *base* class, `AccessorEndowedClass`, from which any class exposing an accessor must inherit, is created. The only thing it does is adding a *class* attribute that is expected by Pandas' `_register_accessor` method.

In [ ]:
# export
class AccessorEndowedClass:
    
    _accessors = set()

For instance

In [ ]:
class Car(AccessorEndowedClass):
    
    def __init__(self, weight):
        
        self.weight = weight

@register_accessor('seat', Car)
class SeatAccessor:
    
    def __init__(self, car: Car):
        
        self._confortable: bool = True
    
    @property
    def confortable(self):
        
        return self._confortable

Accessing the class itself

In [ ]:
car = Car(1600)
car.weight

Accessing the *appended* accessor

In [ ]:
car.seat.confortable

# Assorted

A function to turn an integer into a roman number. Taken from [here](https://www.w3resource.com/python-exercises/class-exercises/python-class-exercise-1.php).

In [ ]:
# export

# taken from https://www.w3resource.com/python-exercises/class-exercises/python-class-exercise-1.php
def int_to_roman(num: int) -> str:
    """
    Returns an integer number in roman format.
    """
    
    val = [
        1000, 900, 500, 400,
        100, 90, 50, 40,
        10, 9, 5, 4,
        1
    ]
    syb = [
        "M", "CM", "D", "CD",
        "C", "XC", "L", "XL",
        "X", "IX", "V", "IV",
        "I"
    ]
    roman_num = ''
    i = 0
    while num > 0:
        for _ in range(num // val[i]):
            roman_num += syb[i]
            num -= val[i]
        i += 1
    return roman_num

assert int_to_roman(12) == 'XII'
assert int_to_roman(9) == 'IX'

In [ ]:
print(int_to_roman(53))

# Files

## `dict_to_yaml`

A function to write a `dict`ionary to a YAML file with some formatting.

In [ ]:
# export

def dict_to_yaml(d: dict, output_file: Union[str, pathlib.Path]) -> None:
    
    yaml = ruamel.yaml.YAML()
    yaml.indent(sequence=4, offset=2)

    with open(output_file, 'w') as f:

        yaml.dump(d, f)

In [ ]:
yaml_file = '_test.yaml'
d = {'foo': 1, 'ouch': 'psss'}
dict_to_yaml(d, yaml_file)

In [ ]:
%pycat {yaml_file}

## `yaml_to_dict`

In [ ]:
# export

def yaml_to_dict(file: Union[str, pathlib.Path]) -> dict:

    with open(file) as f:

        d = yaml.load(f, Loader=yaml.FullLoader)
        
    return d

In [ ]:
yaml_to_dict(yaml_file)

In [ ]:
%rm {yaml_file}

## `write_multiple_categories`

In [ ]:
# export

def write_multiple_categories(
        category_questions: Dict[str, List[dict]], pictures_base_directory: str, output_file: str = 'out.yaml') -> None:
    """
    Writes a file suitable as input to `gift-wrapper`.

    Parameters
    ----------
    category_questions : dict
        Every key is the name of a category, and every value is a list of questions
        (every question is itself a dictionary).
    pictures_base_directory : str
        The "pictures base directory" parameter that must be passed to `gift-wrapper`
    output_file : str
        Output file

    """

    file = dict()
    file['pictures base directory'] = pictures_base_directory
    file['categories'] = []

    for category_name, questions in category_questions.items():

        file['categories'].append({'name': category_name, 'questions': questions})
    
    dict_to_yaml(file, output_file)

We build a dictionary with questions belonging different categories. This might be read from a *YAML* file.

In [ ]:
category_questions_example = {
    ('Category A', 'Category B'): [
        {
            'class': 'MultipleChoice',
            'statement': 'Compute the entropy....\n',
            'feedback': 'We just need to...\n',
            'time': '3',
            'answers': {
                'perfect': '1',
                'wrong': ['0', '2']
            },
            'name': 'Entropy of a random variable I'
        }
    ],
    'Category C': [
        {
            'class': 'Numerical',
            'statement': 'Compute the average....\n',
            'feedback': 'In order to...\n',
            'time': '3',
            'solution': {
                'value': 3.1,
                'error': '10%'
            }, 
            'name': 'Mean energy I'
        }
    ]
}
category_questions_example

In [ ]:
output_file = '_output.yaml'
write_multiple_categories(category_questions_example, 'pics', output_file)

In [ ]:
%cat {output_file}

In [ ]:
!rm {output_file}

## File system

A small utility function to derive a file path by appending something to the name (and *only* to the name).

In [ ]:
# export

def supplement_file_name(file: Union[str, pathlib.Path], sup: str) -> pathlib.Path:
    """
    Adds a string between the file name in a path and the suffix.

    Parameters
    ----------
    file : str
        File name
    sup : str
        String to be added

    Returns
    -------
    out: pathlib.Path
        "Supplemented" file

    """

    file = pathlib.Path(file)

    # the `suffix` is incorporated into the file name
    return file.with_name(file.stem + f'_{sup}' + file.suffix)

assert supplement_file_name('/a/b/quixote.tex', 'foo') == pathlib.Path('/a/b/quixote_foo.tex')

In [ ]:
supplement_file_name('/a/b/picture.tex', '1')

# Question-related

## `add_name`

Adds a name to every question in a list based on a pattern. It is meant to distinctly name different versions of the same question. Notice that, for this function, a *question* is simply a dictionary (no checks are performed).

In [ ]:
# export

def add_name(questions: List[dict], base_name: str) -> List[dict]:
    """
    Adds a name to every question based on a pattern.

    Parameters
    ----------
    questions : list
        List of questions; every question is a dictionary.
    question_base_name : str
        All the questions will be given this name and a different (Roman) number.

    Returns
    -------
    out: list
        List with the same questions after adding the corresponding name to each one.

    """

    res = []

    for i_q, q in enumerate(questions):

        res.append({**q, 'name': f'{base_name} {int_to_roman(i_q + 1)}'})

    return res

assert add_name([{'k1': 'aa', 'k2': 1}, {'k3': 'pi', 'foo': 'variance'}], 'Viterbi') == [
    {'k1': 'aa', 'k2': 1, 'name': 'Viterbi I'}, {'k3': 'pi', 'foo': 'variance', 'name': 'Viterbi II'}]

In [ ]:
add_name([{'k1': 'aa', 'k2': 1}, {'k3': 'pi', 'foo': 'variance'}], 'base')

---
Get wrong solutions from the correct one

In [ ]:
# export

def wrong_numerical_solutions_from_correct_one(
    solution: float, n: int, min_sep: float, max_sep: float, lower_bound: float, upper_bound: float,
    precision: int = 4, to_str: bool = True, bin_width: Optional[float] = None, unique: bool = False,
    prng: np.random.RandomState = np.random.RandomState(42)) -> Union[List[float], List[str]]:
    """
    Generates random numerical wrong answers given the correct one.


    Parameters
    ----------
    solution: float
        The actual solution.
    n: int
        The number of wrong solutions.
    min_sep: float
        Minimum separation.
    max_sep: float
        Maximum separation.
    lower_bound: float
        A lower bound on the returned numbers.
    upper_bound: float
        A upper bound on the returned numbers.
    precision: int
        The number of decimal places.
    to_str: bool, optional
        If True, every element in the result will be converted to a string.
    bin_width: float, optional
        The granularity on the answers: every one will be a multiple of this parameter.
    unique: bool, optional
        If True, all the answers will be different.
    prng: RandomState, optional
        A pseudo-random numbers generator.

    Returns
    -------
    out: list of float, or list of str
        The random wrong solutions.

    """
    
    max_iterations = 1_000
    
    # in the current implementation all the answers are generated by pivoting around the solution; in order to prevent the correct solution being approximately in the middle, more wrong answers than requested are generated and, in the end, the requested number is randomly picked
    n_generated = n + 2
    
    assert (solution - min_sep > lower_bound) or (solution + min_sep < upper_bound)
    
    if bin_width:
        
        assert bin_width < (max_sep - min_sep)
    
    res = []
    
    # this acts as "pivot" to get to new values around
    current = solution
    
    for i in range(max_iterations):
        
        if bin_width:
        
            candidates = np.arange(min_sep, max_sep, bin_width)
            steps = prng.choice(candidates, size=2).round(precision)
        
        else:
            
            # one number to be added to the `current` solution and one to be subracted
            steps = prng.uniform(min_sep, max_sep, size=2)

        next_values = [v.round(precision)
                       for v in [current + steps[0], current - steps[1]] if lower_bound < v < upper_bound]
        if unique:
            
            next_values = [e for e in next_values if e not in set(res)]
        
        res.extend(next_values)
        
        # TODO: `current` *should* be modified so that the results are less centered around the correct solution (it reveals information!!) but with the current implementation it's hard to do while honoring the minimum and maximum separations
#         # this sort of adds entropy
#         current = np.median(res + [solution])

        if len(res) >= n:
        
            break
            
    else:
        
        raise Exception(f"Parameters are too much constraining, can't find the solution after {max_iterations}")
    
    if to_str:
        
        res = [str(e) for e in res]
    
#     return res[:n]
    return prng.choice(res, n, replace=False).tolist()

In [ ]:
wrong_numerical_solutions_from_correct_one(solution=0.8, n=10, min_sep=0.1, max_sep=0.5, lower_bound=0.2, upper_bound=2, prng = np.random.RandomState(42))

Numbers instead of strings

In [ ]:
wrong_numerical_solutions_from_correct_one(
    solution=0.8, n=4, min_sep=0.2, max_sep=0.5, lower_bound=0.2, upper_bound=0.9, to_str=False,
    prng = np.random.RandomState(42))

If the solution cannot be *anything*, you might want to pass a `bin_width` so that it doesn't immediately stand out from the wrong answers,

In [ ]:
wrong_numerical_solutions_from_correct_one(
    solution=0.8, n=8, min_sep=0.1, max_sep=2, lower_bound=0.2, upper_bound=2, to_str=False, bin_width=0.1,
    prng = np.random.RandomState(42))

If duplicates must be avoided `unique` can be set to `True`,

In [ ]:
wrong_numerical_solutions_from_correct_one(
    solution=0.8, n=8, min_sep=0.1, max_sep=2, lower_bound=0.2, upper_bound=2, to_str=False, bin_width=0.1,
    unique=True, prng = np.random.RandomState(42))

In [ ]:
# hide

import nbdev.export
nbdev.export.notebook2script('02_util.ipynb')